In [ ]:
import training

In [ ]:
training.model_summary()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs
%reload_ext tensorboard

In [ ]:
from FoodRatingDataSet import FoodRatingDataset

dataset = FoodRatingDataset()
data = dataset.data(32)
model = training.create_model(9)
training.train(data, model)

In [ ]:
from TestDataSet import TestDataSet

dataset = TestDataSet()
data = dataset.data()
model = training.create_model(5)
training.train(data, model)